In [12]:
import os
import sys
from google.colab import drive

# 1. Připojení Google Drive
drive.mount('/content/drive')

# 2. Nastavení hlavních cest
BASE_PROJECT_PATH = "/content/drive/MyDrive/BUP_Video_Classification"
SRC_PATH = os.path.join(BASE_PROJECT_PATH, "src")
DATA_PATH = os.path.join(BASE_PROJECT_PATH, "data")

# 3. Přidání složky 'src' do sys.path
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Složka {SRC_PATH} přidána do sys.path")

# 4. Instalace závislostí
REQUIREMENTS_PATH = os.path.join(BASE_PROJECT_PATH, "requirements.txt")
if os.path.exists(REQUIREMENTS_PATH):
    print("Instaluji závislosti...")
    !pip install -r "$REQUIREMENTS_PATH"
else:
    print("VAROVÁNÍ: Soubor requirements.txt nenalezen.")

print("\n--- Prostředí je připraveno ---")
print(f"Aktuální pracovní složka: {os.getcwd()}")
print(f"Složka s moduly: {SRC_PATH}")
print(f"Složka s daty: {DATA_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Složka /content/drive/MyDrive/BUP_Video_Classification/src přidána do sys.path
Instaluji závislosti...

--- Prostředí je připraveno ---
Aktuální pracovní složka: /content
Složka s moduly: /content/drive/MyDrive/BUP_Video_Classification/src
Složka s daty: /content/drive/MyDrive/BUP_Video_Classification/data


In [16]:
import src.data_preparation as dp
import importlib

importlib.reload(dp)

# --- FÁZE 1: PŘÍPRAVA DAT ---

# 1. Seznam tříd, které chceme použít
selected_classes_list = [
    'ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'Basketball',
    'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BrushingTeeth',
    'CleanAndJerk', 'CliffDiving', 'CuttingInKitchen', 'Fencing', 'GolfSwing'
]

# 2. Volání funkce pro vytvoření mini-datasetu
dp.create_mini_dataset(
    base_data_path=DATA_PATH,
    selected_classes=selected_classes_list,
    full_dataset_folder="UCF101_full/UCF-101"
)

# 3. Volání funkce pro vytvoření splitů
dp.create_group_splits(base_data_path=DATA_PATH)

print("\nFáze 1 (Příprava dat) je kompletní.")


--- Spouštím create_mini_dataset ---
Zdroj: /content/drive/MyDrive/BUP_Video_Classification/data/UCF101_full/UCF-101
Cíl: /content/drive/MyDrive/BUP_Video_Classification/data/UCF101_mini
Vytvořena složka: /content/drive/MyDrive/BUP_Video_Classification/data/UCF101_mini


Kopírování tříd: 100%|██████████| 15/15 [00:39<00:00,  2.62s/it]

Zkopírováno 1986 nových souborů.
--- Kopírování pro mini-dataset dokončeno ---

--- Spouštím create_group_splits ---
Načítám soubory z: /content/drive/MyDrive/BUP_Video_Classification/data/UCF101_mini
Nalezeno celkem 1986 videí.
Počet trénovacích videí: 1585
Počet testovacích videí: 401
Trénovací data uložena do: /content/drive/MyDrive/BUP_Video_Classification/data/train_split.csv
Testovací data uložena do: /content/drive/MyDrive/BUP_Video_Classification/data/test_split.csv
--- Vytvoření splitů dokončeno ---

Fáze 1 (Příprava dat) je kompletní.
Zkontrolujte složku 'data' na vašem GDrive. Měli byste vidět 'train_split.csv' a 'test_split.csv'.
